In [1]:


import ipyparallel as ipp
c = ipp.Client()
c.ids



[0, 1, 2, 3, 4, 5, 6, 7]

In [2]:
%%px
from mpi4py import MPI
comm = MPI.COMM_WORLD
print('Rank {} of {} is alive.'.format(comm.Get_rank(), comm.Get_size()))

[stdout:0] Rank 0 of 8 is alive.
[stdout:1] Rank 1 of 8 is alive.
[stdout:2] Rank 2 of 8 is alive.
[stdout:3] Rank 3 of 8 is alive.
[stdout:4] Rank 4 of 8 is alive.
[stdout:5] Rank 5 of 8 is alive.
[stdout:6] Rank 6 of 8 is alive.
[stdout:7] Rank 7 of 8 is alive.


In [3]:
%%px
lx = 420
ly = 180
d = 40

In [53]:
%%px
import numpy as np
from mpi4py import MPI
comm = MPI.COMM_WORLD
size = comm.Get_size()
gcd = np.gcd(np.arange(size // 2), size)
gcd_sorted = np.sort(gcd)
cartesian3d = comm.Create_cart(dims = [gcd_sorted[-2], size / gcd_sorted[-2]], periods =[True, True],reorder=False)
rank = comm.Get_rank()
coord3d = cartesian3d.Get_coords(rank)
print ("In 3D topology, Processor ",rank, " has coordinates ",coord3d)

[stdout:0] In 3D topology, Processor  0  has coordinates  [0, 0]
[stdout:1] In 3D topology, Processor  1  has coordinates  [0, 1]
[stdout:2] In 3D topology, Processor  2  has coordinates  [0, 2]
[stdout:3] In 3D topology, Processor  3  has coordinates  [0, 3]
[stdout:4] In 3D topology, Processor  4  has coordinates  [1, 0]
[stdout:5] In 3D topology, Processor  5  has coordinates  [1, 1]
[stdout:6] In 3D topology, Processor  6  has coordinates  [1, 2]
[stdout:7] In 3D topology, Processor  7  has coordinates  [1, 3]


In [21]:
%%px
n_local_x = lx // size
n_local_y = ly // size
if rank == size - 1:
    n_local_x = lx - n_local_x * (size - 1)
    n_local_y = ly - n_local_y * (size - 1)
print(n_local_x, n_local_y)

[stdout:0] 52 22
[stdout:1] 52 22
[stdout:2] 52 22
[stdout:3] 52 22
[stdout:4] 52 22
[stdout:5] 52 22
[stdout:6] 52 22
[stdout:7] 56 26


In [27]:
%%px
left, right = cartesian3d.Shift(direction = 0,disp=1)
print('Process ',rank, ' coordinates ', coord3d, ' with neighbors in x', left, right)

bottom, top = cartesian3d.Shift(direction = 1,disp=1)
print('Process ',rank, ' coordinates ', coord3d, ' with neighbors in y ', top, bottom)

[stdout:0] 
Process  0  coordinates  [0, 0]  with neighbors in x 4 4
Process  0  coordinates  [0, 0]  with neighbors in y  1 3
[stdout:1] 
Process  1  coordinates  [0, 1]  with neighbors in x 5 5
Process  1  coordinates  [0, 1]  with neighbors in y  2 0
[stdout:2] 
Process  2  coordinates  [0, 2]  with neighbors in x 6 6
Process  2  coordinates  [0, 2]  with neighbors in y  3 1
[stdout:3] 
Process  3  coordinates  [0, 3]  with neighbors in x 7 7
Process  3  coordinates  [0, 3]  with neighbors in y  0 2
[stdout:4] 
Process  4  coordinates  [1, 0]  with neighbors in x 0 0
Process  4  coordinates  [1, 0]  with neighbors in y  5 7
[stdout:5] 
Process  5  coordinates  [1, 1]  with neighbors in x 1 1
Process  5  coordinates  [1, 1]  with neighbors in y  6 4
[stdout:6] 
Process  6  coordinates  [1, 2]  with neighbors in x 2 2
Process  6  coordinates  [1, 2]  with neighbors in y  7 5
[stdout:7] 
Process  7  coordinates  [1, 3]  with neighbors in x 3 3
Process  7  coordinates  [1, 3]  with neig

In [34]:
%%px
if coord3d[0] == 0:
    print(coord3d)

[stdout:0] [0, 0]
[stdout:1] [0, 1]
[stdout:2] [0, 2]
[stdout:3] [0, 3]


In [36]:
%%px

n_local_x = lx // size
n_local_y = ly // size
if rank == size - 1:
    n_local_x = lx - n_local_x * (size - 1)
    n_local_y = ly - n_local_y * (size - 1)
    
a = np.ones((n_local_x, n_local_y))
print(a.shape)

[stdout:0] (52, 22)
[stdout:1] (52, 22)
[stdout:2] (52, 22)
[stdout:3] (52, 22)
[stdout:4] (52, 22)
[stdout:5] (52, 22)
[stdout:6] (52, 22)
[stdout:7] (56, 26)


In [91]:
%%px
def x_in_process(coord3d, x_coord, lx, processes_in_x):
    lower = coord3d[0] * (lx // processes_in_x)
    upper = (coord3d[0] + 1) * (lx // processes_in_x) - 1 if not coord3d[0] == processes_in_x - 1 else lx
    return lower <= x_coord <= upper

def y_in_process(coord3d, y_coord, ly, processes_in_y):
    lower = coord3d[1] * (ly // processes_in_y)
    upper = (coord3d[1] + 1) * (ly // processes_in_y) - 1
    return lower <= y_coord <= upper
x_in_process(coord3d, lx -1, lx-1, gcd_sorted[-2])
#y_in_process(coord3d, 179, ly, size / gcd_sorted[-2])

[stdout:0] 419 0 208 [0, 0]
[stdout:1] 419 0 208 [0, 1]
[stdout:2] 419 0 208 [0, 2]
[stdout:3] 419 0 208 [0, 3]
[stdout:4] 419 209 419 [1, 0]
[stdout:5] 419 209 419 [1, 1]
[stdout:6] 419 209 419 [1, 2]
[stdout:7] 419 209 419 [1, 3]


Out[0:85]: False

Out[1:85]: False

Out[2:85]: False

Out[3:85]: False

Out[4:85]: True

Out[5:85]: True

Out[6:85]: True

Out[7:85]: True

In [86]:
%%px
size

Out[0:80]: 8

Out[1:80]: 8

Out[2:80]: 8

Out[3:80]: 8

Out[4:80]: 8

Out[5:80]: 8

Out[6:80]: 8

Out[7:80]: 8